# Data Preparation

### Task
Parse the json **nyc_geo.json** into the dataframe with the following columns:
- Borough
- Neighborhood
- Latitude
- Longitude

In [20]:
import json
import pandas as pd
import os
import requests
import numpy as np
import sqlite3
from sqlite3 import Error
import matplotlib.pyplot as plt

In [21]:
with open('data/nyc_geo.json', 'r') as f:
    nyc_geo = json.load(f)

In [ ]:
nyc_geo

In [23]:
df_nyc_geo = pd.json_normalize(nyc_geo)
df_features = pd.json_normalize(nyc_geo['features'])

In [79]:
df_nyc_geo

,type,totalFeatures,features,bbox,crs.type,crs.properties.name
0,FeatureCollection,306,"[{'type': 'Feature', 'id': 'nyu_2451_34572.1',...","[-74.2492599487305, 40.5033187866211, -73.7061...",name,urn:ogc:def:crs:EPSG::4326


In [80]:
df_features
# for i in df_features['properties.name']:
#     print(i)

,type,id,geometry_name,geometry.type,geometry.coordinates,properties.name,properties.stacked,properties.annoline1,properties.annoline2,properties.annoline3,properties.annoangle,properties.borough,properties.bbox
0,Feature,nyu_2451_34572.1,geom,Point,"[-73.84720052054902, 40.89470517661]",Wakefield,1,Wakefield,None,None,0.0,Bronx,"[-73.84720052054902, 40.89470517661, -73.84720..."
1,Feature,nyu_2451_34572.2,geom,Point,"[-73.82993910812398, 40.87429419303012]",Co-op City,2,Co-op,City,None,0.0,Bronx,"[-73.82993910812398, 40.87429419303012, -73.82..."
2,Feature,nyu_2451_34572.3,geom,Point,"[-73.82780644716412, 40.887555677350775]",Eastchester,1,Eastchester,None,None,0.0,Bronx,"[-73.82780644716412, 40.887555677350775, -73.8..."
3,Feature,nyu_2451_34572.4,geom,Point,"[-73.90564259591682, 40.89543742690383]",Fieldston,1,Fieldston,None,None,0.0,Bronx,"[-73.90564259591682, 40.89543742690383, -73.90..."
4,Feature,nyu_2451_34572.5,geom,Point,"[-73.9125854610857, 40.890834493891305]",Riverdale,1,Riverdale,None,None,0.0,Bronx,"[-73.9125854610857, 40.890834493891305, -73.91..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,Feature,nyu_2451_34572.302,geom,Point,"[-74.00011136202637, 40.75665808227519]",Hudson Yards,2,Hudson,Yards,None,0.0,Manhattan,"[-74.00011136202637, 40.75665808227519, -74.00..."
302,Feature,nyu_2451_34572.303,geom,Point,"[-73.80553002968718, 40.58733774018741]",Hammels,1,Hammels,None,None,0.0,Queens,"[-73.80553002968718, 40.58733774018741, -73.80..."
303,Feature,nyu_2451_34572.304,geom,Point,"[-73.76596781445627, 40.611321691283834]",Bayswater,1,Bayswater,None,None,0.0,Queens,"[-73.76596781445627, 40.611321691283834, -73.7..."
304,Feature,nyu_2451_34572.305,geom,Point,"[-73.94563070334091, 40.756091297094706]",Queensbridge,1,Queensbridge,None,None,0.0,Queens,"[-73.94563070334091, 40.756091297094706, -73.9..."


In [96]:
df_data = df_features[ ['properties.borough', 'properties.name'] ]
df_data = df_data.rename(columns={'properties.borough': 'borough', 'properties.name': 'neighborhood'})
df_data['latitude'] = df_features['geometry.coordinates'].str[1]
df_data['longitude'] = df_features['geometry.coordinates'].str[0]

In [100]:
df_data

,borough,neighborhood,latitude,longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
...,...,...,...,...
301,Manhattan,Hudson Yards,40.756658,-74.000111
302,Queens,Hammels,40.587338,-73.805530
303,Queens,Bayswater,40.611322,-73.765968
304,Queens,Queensbridge,40.756091,-73.945631


In [103]:
df_data.to_csv('data/neighborhood.csv',sep=';')

### Task
Use different data sources and APIs to collect information about the neigborhoods that can be used for segmentation.

### SQL Lite Database for storing the data captured from the APIs

In [6]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [7]:
connection = create_connection("data/midtermproject.sqlite")

Connection to SQLite DB successful


#### Create the required tables

In [133]:
# 1. CREATE poi_info table

#drop_poi_info = "DROP TABLE poi_info;"
#execute_query(connection, drop_poi_info)  

create_poi_into_table = """
CREATE TABLE IF NOT EXISTS poi_info (
    api TEXT NOT NULL,
    poi_id TEXT NOT NULL,
    name TEXT NOT NULL,
    address TEXT,
    province TEXT,
    city TEXT,
    country TEXT,
    postcode TEXT,
    timezone TEXT,
    latitude REAL,
    longitude REAL,
    phone TEXT,
    fax TEXT,
    email TEXT,
    social_media TEXT,
    is_closed TEXT,
    url TEXT,
    image_url TEXT,
    review_count INTEGER,
    rating REAL,
    price_level TEXT,
    PRIMARY KEY (api, poi_id)
);
"""

execute_query(connection, create_poi_into_table)  

Query executed successfully


In [135]:
# 2. CREATE poi_category table

#drop_poi_category = "DROP TABLE poi_category;"
#execute_query(connection, drop_poi_category)  

create_poi_category_table = """
CREATE TABLE IF NOT EXISTS poi_category (
    api TEXT NOT NULL,
    poi_id TEXT NOT NULL,
    neighborhood TEXT NOT NULL,
    category TEXT NOT NULL ,
    PRIMARY KEY (api, poi_id, category)
    FOREIGN KEY (api, poi_id) REFERENCES poi_info(api, poi_id)
);
"""

execute_query(connection, create_poi_category_table)  

Query executed successfully


In [10]:
# 3. CREATE poi_opening_hours table

#drop_poi_opening_hours = "DROP TABLE poi_opening_hours;"
#execute_query(connection, drop_poi_opening_hours)  

create_poi_opening_hours_table = """
CREATE TABLE IF NOT EXISTS poi_opening_hours (
    api TEXT NOT NULL,
    poi_id TEXT NOT NULL,
    day INTEGER NOT NULL,
    open_time INTEGER,
    close_time INTEGER,
    PRIMARY KEY (api, poi_id, day)
    FOREIGN KEY (api, poi_id) REFERENCES poi_info(api, poi_id)
);
"""

execute_query(connection, create_poi_opening_hours_table)  

Query executed successfully


In [11]:
# 4. CREATE poi_hours_popular table

#drop_poi_hours_popular = "DROP TABLE poi_hours_popular;"
#execute_query(connection, drop_poi_hours_popular)  

create_poi_hours_popular_table = """
CREATE TABLE IF NOT EXISTS poi_hours_popular (
    api TEXT NOT NULL,
    poi_id TEXT NOT NULL,
    day INTEGER NOT NULL,
    open_time INTEGER NOT NULL,
    close_time INTEGER,
    PRIMARY KEY (api, poi_id, day, open_time)
    FOREIGN KEY (api, poi_id) REFERENCES poi_info(api, poi_id)
);
"""

execute_query(connection, create_poi_hours_popular_table)  

Query executed successfully


#### Functions to insert records

In [136]:
def insert_poi_info_reccord(connection, record):
    insert_poi_info = """
    INSERT INTO
        poi_info (api, poi_id, name, address, province, city, country, postcode, timezone, latitude, longitude, phone, fax, email, social_media, is_closed, url, image_url, review_count, rating, price_level)
    VALUES
        (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);
    """
    
    cursor = connection.cursor()
    try:
        cursor.execute(insert_poi_info, record)
        connection.commit()
        #print("Query executed successfully")
    except Error as e:
        pass
        #print(f"The error '{e}' occurred")
    
    #execute_query(connection, create_users)  

In [137]:
def insert_poi_category_record(connection, record):
    insert_poi_category = """
    INSERT INTO
        poi_category (api, poi_id, neighborhood, category)
    VALUES
        (?,?,?,?);
    """
    
    cursor = connection.cursor()
    try:
        cursor.execute(insert_poi_category, record)
        connection.commit()
        #print("Query executed successfully")
    except Error as e:
        pass
        #print(f"The error '{e}' occurred")

In [66]:
def insert_poi_opening_hours_record(connection, record):
    insert_poi_opening_hours = """
    INSERT INTO
        poi_opening_hours (api, poi_id, day, open_time, close_time)
    VALUES
        (?,?,?,?,?);
    """
    
    cursor = connection.cursor()
    try:
        cursor.execute(insert_poi_opening_hours, record)
        connection.commit()
        #print("Query executed successfully")
    except Error as e:
        pass
        #print(f"The error '{e}' occurred")

In [67]:
def insert_poi_hours_popular_record(connection, record):
    insert_poi_hours_popular = """
    INSERT INTO
        poi_hours_popular (api, poi_id, day, open_time, close_time)
    VALUES
        (?,?,?,?,?);
    """
    
    cursor = connection.cursor()
    try:
        cursor.execute(insert_poi_hours_popular, record)
        connection.commit()
        #print("Query executed successfully")
    except Error as e:
        pass
        #print(f"The error '{e}' occurred")

#### Get the restaurants information from Foursquare API

In [18]:
# set the keys
foursquare_api_key = os.environ["FOURSQUARE_API_KEY"]
yelp_client_id = os.environ["YELP_CLIENT_ID"]
yelp_api_key = os.environ["YELP_API_KEY"]
google_api_key = os.environ["GOOGLE_API_KEY"]

In [138]:
# setup the parameters

# 10 POIs selected 
# Restaurant
# Bar
# Gym
# Parking
# Health Centre
# Bookstore
# Bank
# Bus Station
# Cafe
# Cinema

categories = [{'foursquare':'13065', 'yelp': 'restaurants', 'google': 'restaurant', 'poi_system': 'restaurants'},
              {'foursquare':'13003', 'yelp': 'bars', 'google': 'bars', 'poi_system': 'bars'},
              {'foursquare':'18021', 'yelp': 'gyms', 'google': 'gyms', 'poi_system': 'gyms'},
              {'foursquare':'19020', 'yelp': 'parking', 'google': 'parkings', 'poi_system': 'parking'},
              {'foursquare':'15000', 'yelp': 'health', 'google': 'health', 'poi_system': 'health'},
              {'foursquare':'17018', 'yelp': 'bookstores', 'google': 'book_stores', 'poi_system': 'bookstores'},
              {'foursquare':'11045', 'yelp': 'banks', 'google': 'banks', 'poi_system': 'banks'},
              {'foursquare':'19042', 'yelp': 'busstations', 'google': 'bus_stations', 'poi_system': 'busstations'},
              {'foursquare':'13034', 'yelp': 'Cafes', 'google': 'cafes', 'poi_system': 'cafes'},
              {'foursquare':'10024', 'yelp': 'Cinema', 'google': 'movie_theaters', 'poi_system': 'cinemas'}
             ]

radius = '300'

In [139]:
def get_foursquare_place(ll, radius, categories):
    url = 'https://api.foursquare.com/v3/places/search?'
    url = url + 'll='+ll
    url = url + '&radius='+radius
    url = url + '&categories='+categories
    url = url + '&fields=fsq_id,name,geocodes,location,categories,related_places,timezone,description,tel,fax,email,website,social_media,hours,hours_popular,rating,stats,popularity,price,menu,photos,tips,tastes'
    url = url + '&limit=50'

    headers = {
        'Accept': 'application/json',
        'Authorization': foursquare_api_key
    }

    res = requests.request("GET", url, headers=headers)

    return res.json()

In [143]:
def collect_foursquare_place(ll, radius, neighborhood, categories):
    for category in categories:
        result = get_foursquare_place(ll, radius, category['foursquare'])
        
        if 'results' in result:

            for i in result['results']:
                record = ['foursquare']
                record.append(i['fsq_id'])
                record.append(i['name'])
                record.append(i['location']['formatted_address'])
                record.append(i['location']['region'])
                record.append(i['location']['locality'])
                record.append(i['location']['country'])
                record.append(i['location']['postcode'] if 'postcode' in i['location'] else '')
                record.append(i['timezone'] if 'timezone' in i else '')
                record.append(float(i['geocodes']['main']['latitude']) if 'main' in i['geocodes'] else '')
                record.append(float(i['geocodes']['main']['longitude']) if 'main' in i['geocodes'] else '')
                record.append(i['tel'] if 'tel' in i else '')
                record.append('')
                record.append(i['email'] if 'email' in i else '')
                record.append(str(i['social_media'] if 'social_media' in i else '') )
                record.append('False' if i['hours']['open_now'] else 'True')
                record.append(i['website'] if 'website' in i else '')
                record.append('')
                if 'stats' in i:
                    if 'total_ratings' in i['stats']:
                        record.append(i['stats']['total_ratings'])
                    else:
                        record.append('')
                else:
                    record.append('')
                record.append(i['rating'] if 'rating' in i else '')
                record.append(i['price'] if 'price' in i else '')

                insert_poi_info_reccord(connection, tuple(record))



                record = ('foursquare', i['fsq_id'], neighborhood, category['poi_system'])
                insert_poi_category_record(connection, record)



                if 'hours' in i:
                    if 'regular' in i['hours']:
                        for h in i['hours']['regular']:
                            record = ('foursquare', i['fsq_id'], h['day'], int(h['open']), int(h['close']))
                            insert_poi_opening_hours_record(connection, record)



                if 'hours_popular' in i:
                    for h in i['hours_popular']:
                        record = ('foursquare', i['fsq_id'], h['day'], int(h['open']), int(h['close']))
                        insert_poi_hours_popular_record(connection, record)

In [164]:
for i in df_data.iterrows():
    ll = str(i[1][2])+','+str(i[1][3])
    print(ll, i[1][1])
    collect_foursquare_place(ll, radius, i[1][1], categories)

40.64692606658579,-73.94817709920184 Erasmus
40.75665808227519,-74.00011136202637 Hudson Yards
40.58733774018741,-73.80553002968718 Hammels
40.611321691283834,-73.76596781445627 Bayswater
40.756091297094706,-73.94563070334091 Queensbridge
40.61731079252983,-74.08173992211962 Fox Hills


In [165]:
#df_data.iloc[0:50] done
#df_data.iloc[50:100] done
#df_data.iloc[100:150] done
#df_data.iloc[150:200] done
#df_data.iloc[200:250] done
#df_data.iloc[250:300] done
#df_data.iloc[300:] done

### Task

Visualize the neigborhoods in the graph.